In [14]:
import sys
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
df_b = pd.read_csv("SQLdata.csv")
df_w = pd.read_csv("weatherdata.csv")

In [16]:
df_b.head()

,Station,Station Address,Station No.,Status,Time,Stands,Bikes Avail,Spaces Avail,Bank
0,SMITHFIELD NORTH,Smithfield North,42.0,OPEN,1520432286000,30,5,25,1
1,PARNELL SQUARE NORTH,Parnell Square North,30.0,OPEN,1520431893000,20,3,17,1
2,CLONMEL STREET,Clonmel Street,54.0,OPEN,1520432004000,33,19,14,0
3,MOUNT STREET LOWER,Mount Street Lower,56.0,OPEN,1520432361000,40,26,12,0
4,CHRISTCHURCH PLACE,Christchurch Place,6.0,OPEN,1520431786000,20,1,19,0


In [17]:
df_w.head()

,temperature,rain,time
0,8.04,0.005,14/03/2018 00:00
1,7.72,0.105,14/03/2018 03:00
2,7.65,0.260,14/03/2018 06:00
3,7.95,0.235,14/03/2018 09:00
4,8.10,0.475,14/03/2018 12:00


In [18]:
df_b=df_b.drop(['Station','Station Address'],axis =1)

In [19]:
df_bikes= df_b.drop_duplicates()

In [20]:
df_w= df_w.drop_duplicates()

In [21]:
df_weather= df_w.drop_duplicates("time")

In [22]:
import time
pattern='%d/%m/%Y %H:%M'

In [23]:
#There are a little difference between tamp in weather and bike Since the bike time has accuacy in millseconds
TimestampWeather=[]
for index, row in df_weather.iterrows():
    TimestampWeather.append(int(time.mktime(time.strptime(row['time'],pattern))* 1000))

In [24]:
TimestampWeather[0:5]

[1520985600000, 1520996400000, 1521007200000, 1521018000000, 1521028800000]

In [25]:
df_weather["tamp"] = TimestampWeather

/home/wu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
df_weather.head()

,temperature,rain,time,tamp
0,8.04,0.005,14/03/2018 00:00,1520985600000
1,7.72,0.105,14/03/2018 03:00,1520996400000
2,7.65,0.260,14/03/2018 06:00,1521007200000
3,7.95,0.235,14/03/2018 09:00,1521018000000
4,8.10,0.475,14/03/2018 12:00,1521028800000


In [28]:
time = []
import datetime
for index, row in df_bikes.iterrows():
    time.append(datetime.datetime.fromtimestamp(int(row["Time"]/1000)).strftime('%d/%m/%Y %H:%M:%S'))
time[0:5]

['07/03/2018 14:18:06',
 '07/03/2018 14:11:33',
 '07/03/2018 14:13:24',
 '07/03/2018 14:19:21',
 '07/03/2018 14:09:46']

In [29]:
df_bikes["human_readible"] = time

/home/wu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
df_bikes.head()

,Station No.,Status,Time,Stands,Bikes Avail,Spaces Avail,Bank,human_readible
0,42.0,OPEN,1520432286000,30,5,25,1,07/03/2018 14:18:06
1,30.0,OPEN,1520431893000,20,3,17,1,07/03/2018 14:11:33
2,54.0,OPEN,1520432004000,33,19,14,0,07/03/2018 14:13:24
3,56.0,OPEN,1520432361000,40,26,12,0,07/03/2018 14:19:21
4,6.0,OPEN,1520431786000,20,1,19,0,07/03/2018 14:09:46


In [31]:
import numpy as np
weather_tamp=[]
weather_rain=[]
weather_temperature = []
weather_time =[]
for index, row in df_bikes.iterrows():
    myNumber = row["Time"]
    closestTime = min(TimestampWeather, key = lambda x : abs(x-myNumber))
    if(abs(myNumber-closestTime) > 60 * 60 * 1000):
        weather_time.append(np.nan)
        weather_rain.append(np.nan)
        weather_temperature.append(np.nan)
        weather_tamp.append(np.nan)
    else:
        weather_tamp.append(closestTime)
        weather_rain.append(df_weather.loc[df_weather['tamp'] == closestTime,'rain'].item())
        weather_temperature.append(df_weather.loc[df_weather['tamp'] == closestTime,'temperature'].item())
        weather_time.append(df_weather.loc[df_weather['tamp'] == closestTime,'time'].item())

In [32]:
df_bikes["weather_tamp"] = pd.Series(weather_tamp).values
df_bikes["weather_rain"] = pd.Series(weather_rain).values
df_bikes["weather_temperature"] =pd.Series(weather_temperature).values
df_bikes["weather_time"]=pd.Series(weather_time).values

/home/wu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/wu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/wu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [34]:
df_bikes.head(20)

,Station No.,Status,Time,Stands,Bikes Avail,Spaces Avail,Bank,human_readible,weather_tamp,weather_rain,weather_temperature,weather_time
0,42.0,OPEN,1520432286000,30,5,25,1,07/03/2018 14:18:06,NaN,NaN,NaN,NaN
1,30.0,OPEN,1520431893000,20,3,17,1,07/03/2018 14:11:33,NaN,NaN,NaN,NaN
2,54.0,OPEN,1520432004000,33,19,14,0,07/03/2018 14:13:24,NaN,NaN,NaN,NaN
3,56.0,OPEN,1520432361000,40,26,12,0,07/03/2018 14:19:21,NaN,NaN,NaN,NaN
4,6.0,OPEN,1520431786000,20,1,19,0,07/03/2018 14:09:46,NaN,NaN,NaN,NaN
5,18.0,OPEN,1520432082000,30,0,30,1,07/03/2018 14:14:42,NaN,NaN,NaN,NaN
6,32.0,OPEN,1520432209000,30,22,8,1,07/03/2018 14:16:49,NaN,NaN,NaN,NaN
7,52.0,OPEN,1520431998000,32,32,0,0,07/03/2018 14:13:18,NaN,NaN,NaN,NaN
8,48.0,OPEN,1520432309000,40,38,0,0,07/03/2018 14:18:29,NaN,NaN,NaN,NaN
9,13.0,OPEN,1520432330000,30,26,4,0,07/03/2018 14:18:50,NaN,NaN,NaN,NaN


In [37]:
weather_time.count(np.nan)

72193

In [38]:
df_bikes_clean=df_bikes.dropna(how ='any')

In [39]:
df_bikes_clean.head()

,Station No.,Status,Time,Stands,Bikes Avail,Spaces Avail,Bank,human_readible,weather_tamp,weather_rain,weather_temperature,weather_time
98900,42.0,OPEN,1521068603000,30,10,20,1,14/03/2018 23:03:23,1.521072e+12,0.78,8.45,15/03/2018 00:00
98901,30.0,OPEN,1521068983000,20,20,0,1,14/03/2018 23:09:43,1.521072e+12,0.78,8.45,15/03/2018 00:00
98902,54.0,OPEN,1521068619000,33,2,31,0,14/03/2018 23:03:39,1.521072e+12,0.78,8.45,15/03/2018 00:00
98903,56.0,OPEN,1521068973000,40,6,34,0,14/03/2018 23:09:33,1.521072e+12,0.78,8.45,15/03/2018 00:00
98904,6.0,OPEN,1521069146000,20,4,16,0,14/03/2018 23:12:26,1.521072e+12,0.78,8.45,15/03/2018 00:00


In [40]:
df_bikes_clean.to_csv('historic_mix.csv', index = False)